In [1]:


import os

#os.chdir('/data/zzh/pycharm/GPTICC')
import sys

sys.path.append('./src/')



import torch
from torch.utils.data import DataLoader

import numpy as np
import lmdb
import pickle
from torch.utils.data import Dataset

from models.gpt4protein import gpt2_protein


In [2]:

class ProteinDatasetPre(Dataset):

    def __init__(self, processed_path='./data/human_unreviewed.lmdb',length=None):
        self.processed_path = processed_path
        self.db1 = lmdb.open(
            self.processed_path,
            map_size=1000 * (1024 * 1024 * 1024),  # 1000GB
            create=False,
            subdir=False,
            readonly=True,
            lock=False,
            readahead=False,
            meminit=False,
        )

        with self.db1.begin() as txn:
            self.keys = list(txn.cursor().iternext(values=False))
        if length is not None:
            self.keys = self.keys[:length]

        

    def __getitem__(self, index):
        key = self.keys[index]
        protein_embedd = pickle.loads(self.db1.begin().get(key))
        protein_embedd = torch.tensor(protein_embedd.astype('float32'))
       
        return key.decode(), protein_embedd.unsqueeze(0)

    def __len__(self):
        return len(self.keys)

In [3]:
myset = ProteinDatasetPre()

In [4]:
config = {}
config['patch_size']=8
config['d_model']=768
config['input_dim']=1280
config['num_classes']=2
config['freeze'] = True
config['optimizer'] = 'RAdam'
config['lr'] = 0.0001
config['l2_reg'] = 5e-4
config['if_softmax'] = True
config['lr_step']=1000
config['load_model']= '/data/zzh/gpt2_classification/GPT2'
config['lr_factor']=0.1


weight_decay = config['l2_reg']

model = gpt2_protein(config)
model.load_state_dict(torch.load('./ckpt/gpticc.pth'))

<All keys matched successfully>

In [5]:
model = model.cuda()

In [6]:
test_loader = DataLoader(dataset=myset,
                              batch_size=10240,
                              shuffle=True,
                              num_workers=16,
                              pin_memory=True)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
name_ls = []
out_ls = []

with torch.no_grad():
    for i, batch in enumerate(test_loader):
        print(i)
        a,b = batch
        out = model(b.to(device))
        for out_tensor in out:
            out_ls.append(out_tensor)
        for name in a:
            name_ls.append(name)

        torch.cuda.empty_cache()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [ ]:
out_dict = {}
for i in range(len(name_ls)):
    out_dict[name_ls[i]] = [out_ls[i].detach().cpu().numpy()]

In [ ]:
out_dict

In [ ]:
len(out_dict)

In [ ]:
pred_channel = []
for k, v in out_dict.items():
    if np.argmax(v[0])==0:
        pred_channel.append(k)

In [1]:
len(pred_channel)

NameError: name 'pred_channel' is not defined

In [12]:
pred_channel

['A5X2V1',
 'A0A0G2JR60',
 'A2RUE7',
 'Q9BZ15',
 'A0A1B0GUB6',
 'D2XBF0',
 'B4DH28',
 'B4E3W4',
 'C9JAX7',
 'H0Y7I8',
 'B3KXQ3',
 'H0Y406',
 'A2SX37',
 'A0AAA9YHM0',
 'B4DKD3',
 'K7EQ78',
 'E9PIE8',
 'I3L1T7',
 'A0A1X7SBZ8',
 'A0A7G3W903',
 'O95387',
 'A8K432',
 'Q8WW53',
 'A0A1W2PP26',
 'A0A1B0GU04',
 'A0A2R4LFM3',
 'A0A804HJB6',
 'A0A804HKU0',
 'A0A1B0GV43',
 'A0A7R6V2A9',
 'A0A8I5KVV2',
 'H3BPP0',
 'A0A1B0GUM5',
 'A0A8V8TPE9',
 'E7EV53',
 'F5H522',
 'A0A8I5KVL1',
 'Q53Y30',
 'A0A0S2Z3G7',
 'B2R9S2',
 'Q4W595',
 'A8K177',
 'A0A1W2PQJ9',
 'Q6B4S4',
 'D6RGY7',
 'H0YJ81',
 'A0A1B0GWF7',
 'F5GZ52',
 'A0A1W2PQ81',
 'H7BYG7',
 'S5NU80',
 'H0YAS3',
 'A0A1W2PPY5',
 'A0A6Q8PGT5',
 'Q1WWK6',
 'A0A1W2PQ71',
 'A0A6Q8PGM3',
 'A0A3B3IRL4',
 'F8VR74',
 'A0A6Q8PGY3',
 'E7ETX0',
 'V5QFN8',
 'B4DKC0',
 'A0A1W2PNY7',
 'H3BUQ8',
 'Q59EW6',
 'A0A1B0GW49',
 'Q20BI6',
 'E9PNM2',
 'B3KP17',
 'A0A2R4LFI1',
 'H7BZ66',
 'Q59GW0',
 'B2RDS2',
 'D2KFF9',
 'K7EQ95',
 'Q3SYI1',
 'A0A9L9PXL0',
 'Q6ZMN3',
 'D1MQ08',


In [20]:
pd.DataFrame(pred_channel).to_csv('/data/zzh/gpt2_classification/Classification/result/gpt2_pred_human_channel.csv',index=False)